# 从协议到抽象基类

Python将协议定义为非正式的接口，用于实现多态。因为Python没有interface关键字，按照约定，除了受保护的属性和私有属性不在接口中以外，类实现或继承的公开属性（方法或数据属性），包括特殊方法，都是类的接口。

Python中使用**鸭子类型**：对象的类型无关紧要，只要实现了特定的协议即可。**猴子补丁**则可以用于在动态运行时给类附加方法，以实现某些目的。使用猴子补丁时，打补丁的代码和要打补丁的程序耦合十分紧密，而且往往要处理隐藏和没有文档的部分。

**白鹅类型**：只要cls是抽象基类，即cls的元类是abc.ABCMeta，就可以使用`instance(obj, cls)`。

## 定义一个抽象基类

In [1]:
import abc

class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable):
        """
        从可迭代对象中添加元素。
        """
    
    @abc.abstractmethod
    def pick(self):
        """
        随机删除元素，然后将其返回。
        如果实例为空，这个方法应该抛出'LookupError'。
        """
    
    def loaded(self):
        """
        如果至少有一个元素，返回'True'，否则返回'False'。
        """
        return bool(self.inspect())
    
    def inspect(self):
        """
        返回一个有序元组，由当前元素构成。
        """
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))
    

具体子类可以通过其内部数据结构来更好的实现`loaded`和`inspect`方法。

先使用一个有缺陷的实现糊弄Tombola，来感受抽象基类对接口进行的检查：

In [2]:
class Fake(Tombola):
    def pick(self):
        return 13

In [3]:
Fake

__main__.Fake

In [4]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

错误显示出`Fake`类并没有实现抽象方法`load`，接下来实现一个满足接口规定的子类：

In [5]:
import random

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

`BingoCage`继承了臃肿的`loaded`和`inspect`，但这两个方法都可以在子类覆盖并提供更理想的实现。下面提供一个更好的实现方法：

In [6]:
import random

class LotteryBlower(Tombola):
    def __init__(self, iterable):
        self._balls = list(iterable) # 重新创建iterable的副本而不是直接将self._balls作为iterable的引用。
                                     # 这样可以避免用户自己的数据被修改。
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls) # 在Tombola中的实现是调用了inspect方法，而在这里由于知晓具体内部实现，可以使用更快速的实现手段。
    
    def inspect(self):
        return tuple(sorted(self._balls))

## 使用`register`方法声明虚拟子类

这是白鹅类型的重要特性：即使不继承，也有办法把一个类注册为抽象基类的虚拟子类。这样做时，我们要保证注册的类忠实地实现了抽象基类定义的接口。注册虚拟子类的方法是在抽象基类上调用`register`方法，注册后，`issubclass`和`isinstance`等函数都能识别，但注册的类不会从抽象基类中继承任何方法或属性。再次强调，为了避免错误，我们应当在子类中确保父类的接口都被正确实现，虽然它们能被运行时错误捕获。

下面给出一个例子：

In [7]:
from random import randrange

@Tombola.register # 注册TomboList为Tombola的虚拟子类，注意在Python 3.3及以前的版本中不能当作类装饰器来使用
class TomboList(list): # TomboList是list的真实子类
    def pick(self):
        if self: # 从list中继承了__bool__方法
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))

In [8]:
issubclass(TomboList, Tombola)

True

In [9]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

使用`__mro__`（Method Resolution Order，方法解析顺序）来查看其继承顺序：

In [10]:
TomboList.__mro__

(__main__.TomboList, list, object)

会发现其只列出了“真实的”超类。

Python中，通常使用非正式的协议来实现接口，同时通过鸭子类型来实现多态。通过对抽象基类的简单探究，我们了解了一些关于鸭子类型和白鹅类型的风格的差别。

显示继承抽象基类有一些优点（比如使动态类型检查变得容易了），但记住：不要自己定义抽象基类，除非要构建允许用户拓展的框架。实际使用中，通常是创建现有抽象基类的子类，或者使用现有的抽象基类注册。